In [77]:
import pandas as pd
import pickle
from geopy.distance import geodesic
import numpy as np

# Cargar los modelos y los datos
df_tiendas = pickle.load(open("df_tiendas.pkl", "rb"))
df_eventos = pickle.load(open("df_eventos.pkl", "rb"))
df_actividades = pd.read_pickle("df_actividades.pkl")
cosine_sim_tiendas = pickle.load(open("cos_sim_tiendas.pkl", "rb"))
cosine_sim_eventos = pickle.load(open("cos_sim_eventos.pkl", "rb"))
cosine_sim_actividades = pickle.load(open("cos_sim_actividades.pkl", "rb"))

def obtener_recomendaciones_con_distancia(id, df, id_df, user_location, cosine_sim, radius_km=1):
    if id not in df[id_df].values:
        return []  # O puedes devolver un mensaje de error adecuado
    
    idx = df.index[df[id_df] == id].tolist()[0]
    
    # Prefiltrar tiendas dentro de un cuadrado de lado 2 * radius_km
    lat_min = user_location[0] - radius_km / 111  # Aproximación: 1 grado de latitud ≈ 111 km
    lat_max = user_location[0] + radius_km / 111
    lon_min = user_location[1] - radius_km / (111 * np.cos(np.radians(user_location[0])))
    lon_max = user_location[1] + radius_km / (111 * np.cos(np.radians(user_location[0])))
    
    prefiltered_tiendas = df[
        (df['latitud'] >= lat_min) & (df['latitud'] <= lat_max) &
        (df['longitud'] >= lon_min) & (df['longitud'] <= lon_max)
    ]
    
    # Calcular distancias geodésicas solo para las tiendas prefiltradas
    tiendas_dentro_radio = []
    for i in prefiltered_tiendas.index:
        tienda_location = (prefiltered_tiendas.at[i, 'latitud'], prefiltered_tiendas.at[i, 'longitud'])
        distance = geodesic(user_location, tienda_location).km
        if distance <= radius_km:
            tiendas_dentro_radio.append((i, distance))
    
    if not tiendas_dentro_radio:
        return []  # No hay tiendas dentro del radio especificado
    
    # Calcular similitud del coseno solo para las tiendas dentro del radio
    sim_scores = [(i, cosine_sim[idx][i]) for i, _ in tiendas_dentro_radio]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:5]  # Obtener los 4 más similares
    tienda_indices = [i[0] for i in sim_scores]
    
    recomendaciones = []
    for i in tienda_indices:
        tienda_location = (df.at[i, 'latitud'], df.at[i, 'longitud'])
        distance = geodesic(user_location, tienda_location).km
        tienda_info = df.iloc[i].to_dict()
        tienda_info['distance'] = round(distance, 2)
        recomendaciones.append(tienda_info)
        print(f"Tienda: {df.at[i, 'nombre']}, Distancia: {distance:.2f} km, Similitud: {sim_scores[tienda_indices.index(i)][1]:.2f}")  # Imprimir distancia y similitud
    
    return recomendaciones


# 19321: pardos
# Uso de ejemplo
#9143: La Lucha Sangucheria Criolla
# -12.135105794453583, -77.02212741858906 coordenadas UTEC
#17989: Consultorio Odontologico Brasilerio
#user_location = (-12.130843, -76.982489)  # Ubicación del usuario (para probar puse ubicacion de la tienda)
id = 13  # ID de la tienda
#user_location = (-12.135105794453583, -77.02212741858906)  # Ubicación del usuario (para probar puse ubicacion de la tienda)
user_location = (-12.094456,-77.0798918)
radius_km = 10  # Radio de búsqueda en kilómetros
id_df = 'actividad_id'
print(f'Tienda: {id}, Ubicación: {user_location}, Radio: {radius_km}')
recomendaciones = obtener_recomendaciones_con_distancia(id, df_actividades, id_df, user_location, cosine_sim_actividades, radius_km)
print(recomendaciones)

Tienda: 13, Ubicación: (-12.094456, -77.0798918), Radio: 10
Tienda: Cata de Vinos, Distancia: 8.44 km, Similitud: 0.96
Tienda: Cata de Vinos, Distancia: 8.54 km, Similitud: 0.94
Tienda: Clase de Cocina Peruana, Distancia: 9.42 km, Similitud: 0.18
Tienda: Clase de Cocina Peruana, Distancia: 8.43 km, Similitud: 0.17
[{'actividad_id': 58, 'nombre': 'Cata de Vinos', 'categorias': 'Gastronomía,Experiencia', 'descripcion_breve': 'Descubre los mejores vinos con una cata guiada.', 'latitud': -12.1218, 'longitud': -77.007504, 'direccion': 'Dirección #58-Cata de Vinos', 'calificaciones': 2.5, 'precio': 40, 'fecha_inicio': Timestamp('2024-07-02 00:00:00'), 'fecha_fin': Timestamp('2024-07-05 00:00:00'), 'horario_disponible': ['Lunes 10:00 - 22:00', 'Miércoles 16:00 - 20:00', 'Sábado 14:00 - 24:00', 'Jueves 18:00 - 22:00', 'Viernes 10:00 - 18:00', 'Martes 14:00 - 20:00', 'Domingo 10:00 - 22:00'], 'material_incluido': False, 'target_etario': np.str_('18-40'), 'redes_sociales': 'fb.com/catadevinos', 

In [2]:
# ver todas las categorias de las tiendas
df_tiendas.tail()

,local_id,nombre,categorias,descripcion_breve,latitud,longitud,direccion,imagenes,horario,calificaciones,...,estacionamiento,accesibilidad,target_etario,especialidades,interacciones_acumuladas,visitas_historicas,zona,mejor_hora,redes_sociales,tags
6037,6038,Distribuidora Moisés Cano,comida,Deliciosa comida para todos los gustos.,-12.063492,-77.084214,"Calle los cipreces mz p lote 5, San Miguel",https://picsum.photos/1000/573,Lunes de 08:00-20:00,0.0,...,False,True,18-40,"Postres,Bebidas,Platos variados",1,116,moderna,16:00-18:00,https://facebook.com/distribuidoramoiséscano,"Variedad,Sabor"
6038,6039,Bazar Estefany,"tienda de artículos para el hogar, tienda",Todo lo que necesitas para tu hogar.,-12.063477,-77.084252,"Los Cipreses mz p, San Miguel",https://placekitten.com/500/313,Diario de 08:00-20:00,0.0,...,False,False,25-50,"Decoración,Ofertas,Productos variados",45,34,moderna,20:00-22:00,https://facebook.com/bazarestefany,"Variedad,Conveniencia"
6039,6040,Gaby Burger,"comida rapida, sandwich, hamburguesa, comida, ...",Comida rápida y deliciosa.,-12.063479,-77.084204,"15087, San Miguel",https://dummyimage.com/111x548,Lunes de 08:00-20:00,0.0,...,True,True,25-50,"Hamburguesas clásicas,Bebidas",39,170,moderna,20:00-22:00,https://facebook.com/gabyburger,"Juvenil,Diario"
6040,6041,GRUPO RMH,salud,Servicios de salud y bienestar para toda la fa...,-12.064507,-77.079920,"Calle las Orquideas 110, San Miguel",https://dummyimage.com/368x594,Lunes a Viernes de 08:00-20:00,0.0,...,False,True,18-65,"Exámenes,Tratamientos,Consultas médicas",39,102,moderna,10:00-12:00,https://facebook.com/grupormh,"Confianza,Bienestar"
6041,6042,El Taller de Andrea,tienda,Productos y servicios de calidad.,-12.064183,-77.080339,"Azahares 106, San Miguel",https://placekitten.com/443/944,Sábado de 09:00-18:00,0.0,...,False,False,18-65,"Servicios,Ofertas,Productos variados",31,170,moderna,20:00-22:00,https://facebook.com/eltallerdeandrea,"Variedad,Conveniencia,Calidad"


In [30]:
df_tiendas['categorias'].unique()

array(["['restaurante', 'comida']",
       "['tienda de muebles', 'tienda de artículos para el hogar', 'tienda']",
       "['almacenamiento']", "['lavandería']",
       "['tienda de ropa', 'tienda']", "['electricista']",
       "['comida', 'carnes', 'restaurante']", "['comida', 'bodega']",
       "['salud']", "['parque']", "['gimnasio', 'salud']",
       "['comida', 'tienda de conveniencia', 'tienda']",
       "['comida', 'criolla', 'restaurante']", "['tienda']",
       "['tienda', 'salud']", "['reparación de coches']",
       "['tienda de electrónica', 'tienda']", "['alojamiento']",
       "['licorería', 'tienda']", "['agencia de viajes']",
       "['comida', 'bodega', 'tienda', 'tienda de comestibles o supermercado']",
       "['lavado de coches']", "['comida', 'abarrotes', 'bodega']",
       "['spa']", "['marino', 'comida', 'restaurante', 'cevicheria']",
       "['tienda', 'comida', 'pasteleria', 'panadería']",
       "['comida', 'tienda', 'pasteleria', 'panadería']",
       "['tien

In [76]:
from geopy.distance import geodesic
from difflib import get_close_matches
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from difflib import get_close_matches
import pickle

df_tiendas = pickle.load(open("df_tiendas.pkl", "rb"))
df_interacciones = pd.read_pickle("df_interacciones.pkl")

df_interacciones_locales = pd.read_pickle("df_interacciones_local.pkl")
df_interacciones_eventos = pd.read_pickle("df_interacciones_eventos.pkl")
df_interacciones_actividades = pd.read_pickle("df_interacciones_actividades.pkl")

interaction_matrix_locales = pd.read_pickle("TIENDA_interaction_matrix.pkl")
interaction_matrix_eventos = pd.read_pickle("EVENTO_interaction_matrix.pkl")
interaction_matrix_actividades = pd.read_pickle("ACTIVIDAD_interaction_matrix.pkl")

latent_matrix_locales = pd.read_pickle("TIENDA_latent_matrix.pkl")
latent_matrix_eventos = pd.read_pickle("EVENTO_latent_matrix.pkl")
latent_matrix_actividades = pd.read_pickle("ACTIVIDAD_latent_matrix.pkl")

latent_matrix_locacles_df = pd.read_pickle("TIENDA_latent_matrix_df.pkl")
latent_matrix_eventos_df = pd.read_pickle("EVENTO_latent_matrix_df.pkl")
latent_matrix_actividades_df = pd.read_pickle("ACTIVIDAD_latent_matrix_df.pkl")

item_features_encoded_locales = pd.read_pickle("TIENDA_item_features_encoded.pkl")
item_features_encoded_eventos = pd.read_pickle("EVENTO_item_features_encoded.pkl")
item_features_encoded_actividades = pd.read_pickle("ACTIVIDAD_item_features_encoded.pkl")

item_similarity_locales = np.load("TIENDA_item_similarity.npy")
item_similarity_eventos = np.load("EVENTO_item_similarity.npy")
item_similarity_actividades = np.load("ACTIVIDAD_item_similarity.npy")


def hybrid_recommendation_with_location_and_preference(
    user_id, user_location, user_preference, df, id, 
    interaction_matrix, latent_matrix, latent_matrix_df, item_similarity, item_features_encoded, 
    top_n=4, radius_km=10):
    # Filter stores within the specified radius
    stores_within_radius = []
    for i, row in df.iterrows():
        store_location = (row['latitud'], row['longitud'])
        distance = geodesic(user_location, store_location).km
        if distance <= radius_km:
            stores_within_radius.append((i, distance))  # Store index and distance

    # Filter stores based on user preference matching store categories
    filtered_stores_df = df.loc[[i for i, _ in stores_within_radius]]
    filtered_stores_df = filtered_stores_df[filtered_stores_df['categorias'].str.contains(user_preference, case=False, na=False)]
    
    # Si no encuentra la categoria, buscar por el nombre más cercano y aplicar modelo basado en contenido
    if filtered_stores_df.empty:
        print("No stores match the user preference within the specified radius.")
        
        # buscar coincidencia en el nombre
        store_names = df['nombre'].str.lower().unique()
        store_name = user_preference.lower()
        
        print(f"Searching for close matches to '{store_name}'...")

        
        matches = get_close_matches(store_name, store_names, n=1, cutoff=0.6)
        if matches:
            return obtener_recomendaciones_con_distancia(df[df['nombre'].str.lower() == matches[0]].iloc[0][id], user_location, radius_km=radius_km)
        else:
            print("No close matches found.")
            return []

    # Collaborative Filtering Scores
    if user_id not in latent_matrix_df.index:
        # If the user is new, recommend based on the most popular items in the filtered set
        popular_items = filtered_stores_df[id].value_counts().index[:top_n]
        recommended_stores = filtered_stores_df[filtered_stores_df[id].isin(popular_items)]
        
        # Convert each row to a full dictionary to include all store information
        recommendations_json = recommended_stores.apply(lambda row: row.to_dict(), axis=1).tolist()
        
        return recommendations_json


    # Get the user's latent vector from collaborative filtering
    user_latent_vector = latent_matrix_df.loc[user_id].values.reshape(1, -1)
    cf_scores = np.dot(user_latent_vector, latent_matrix.T).flatten()

    # Content-Based Scores
    user_interactions = df_interacciones[df_interacciones['user_id'] == user_id]['contenido_id']
    if user_interactions.empty:
        # If the user has no interactions, use only content-based scores
        cb_scores = item_similarity.mean(axis=0)
    else:
        # Map user_interactions to item_similarity indices
        user_interaction_indices = [
            item_features_encoded.index.get_loc(tienda_id) 
            for tienda_id in user_interactions if tienda_id in item_features_encoded.index
        ]
        cb_scores = item_similarity[user_interaction_indices].mean(axis=0)

    # Combine CF and CB scores for the filtered set of stores, checking bounds
    hybrid_scores = []
    for tienda_id, distance in stores_within_radius:
        if tienda_id in filtered_stores_df[id].values:
            try:
                idx = interaction_matrix.columns.get_loc(tienda_id)
                # Ensure the index is within bounds for both cf_scores and cb_scores
                if idx < len(cf_scores) and idx < len(cb_scores):
                    score = 0.7 * cf_scores[idx] + 0.3 * cb_scores[idx]
                elif idx < len(cf_scores):
                    score = cf_scores[idx]
                else:
                    score = 0  # Default score if no valid index found
                hybrid_scores.append((tienda_id, score, distance))
            except KeyError:
                # Handle cases where tienda_id is not in the interaction_matrix columns
                #print(f"Store ID {tienda_id} not found in interaction matrix.")
                continue

    # Sort by hybrid score and select top N
    top_recommendations = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:top_n]
    recommended_store_ids = [item[0] for item in top_recommendations]

    # Prepare the JSON-like list of dictionaries for the top recommendations
    recommendations_json = []
    for store_id, score, distance in top_recommendations:
        store_info = df[df[id] == store_id].iloc[0].to_dict()
        recommendations_json.append(store_info)

    # Return the list of JSON-like dictionaries
    return recommendations_json

# Example usage
user_id = 10
user_location = (-12.135105794453583, -77.02212741858906)  # UTEC
user_preference = 'parque'
id = 'local_id' # 'evento_id' or 'actividad_id'
recommendations = hybrid_recommendation_with_location_and_preference(
    user_id, user_location, user_preference, df_tiendas, id,
    interaction_matrix_locales, latent_matrix_locales, latent_matrix_locacles_df, 
    item_similarity_locales, item_features_encoded_locales, top_n=4, radius_km=10)
print(recommendations)

[{'local_id': 19, 'nombre': 'PARQUE BARZOLA', 'categorias': 'parque', 'descripcion_breve': 'Un lugar para relajarse y disfrutar de la naturaleza.', 'latitud': -12.198197, 'longitud': -76.97535719999999, 'direccion': 'R22F+PVC, Sector, Calle 9, Santiago de Surco', 'imagenes': 'https://picsum.photos/286/557', 'horario': 'Sábado de 09:00-18:00', 'calificaciones': 3.5, 'rango_precios': np.str_('Alto'), 'metodos_pago': np.str_('Tarjeta'), 'wifi': True, 'estacionamiento': True, 'accesibilidad': False, 'target_etario': np.str_('25-50'), 'especialidades': 'Zonas de picnic,Juegos infantiles,Áreas verdes', 'interacciones_acumuladas': 7, 'visitas_historicas': 72, 'zona': 'top', 'mejor_hora': np.str_('12:00-14:00'), 'redes_sociales': 'https://facebook.com/parquebarzola', 'tags': 'Relajación,Naturaleza'}, {'local_id': 141, 'nombre': 'Gregorio Mendel Park', 'categorias': 'parque', 'descripcion_breve': 'Un lugar para relajarse y disfrutar de la naturaleza.', 'latitud': -12.1591693, 'longitud': -76.99

In [47]:
from geopy.distance import geodesic
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocesar los nombres de las tiendas
df_tiendas['nombre'] = df_tiendas['nombre'].str.lower()
df_eventos['nombre'] = df_eventos['nombre'].str.lower()
df_actividades['nombre'] = df_actividades['nombre'].str.lower()


# Calcular la matriz TF-IDF con stopwords y tokenización
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_tiendas['nombre'])

tfidf_vectorizer2 = TfidfVectorizer(stop_words='english')
tfidf_matrix_eventos = tfidf_vectorizer2.fit_transform(df_eventos['nombre'])

tfidf_vectorizer3 = TfidfVectorizer(stop_words='english')
tfidf_matrix_actividades = tfidf_vectorizer3.fit_transform(df_actividades['nombre'])

# Guardar el pickle del modelo
pickle.dump(tfidf_vectorizer, open("tfidf_motor_search.pkl", "wb"))
pickle.dump(tfidf_matrix, open("tfidf_motor_search_matrix.pkl", "wb"))
pickle.dump(tfidf_vectorizer2, open("tfidf_motor_search_eventos.pkl", "wb"))
pickle.dump(tfidf_matrix_eventos, open("tfidf_motor_search_matrix_eventos.pkl", "wb"))
pickle.dump(tfidf_vectorizer3, open("tfidf_motor_search_actividades.pkl", "wb"))
pickle.dump(tfidf_matrix_actividades, open("tfidf_motor_search_matrix_actividades.pkl", "wb"))

In [74]:
from geopy.distance import geodesic
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los modelos y los datos
df_tiendas = pickle.load(open("df_tiendas.pkl", "rb"))
motor_search = pickle.load(open("tfidf_motor_search.pkl", "rb"))
motor_search_matrix = pickle.load(open("tfidf_motor_search_matrix.pkl", "rb"))
motor_search_eventos = pickle.load(open("tfidf_motor_search_eventos.pkl", "rb"))
motor_search_matrix_eventos = pickle.load(open("tfidf_motor_search_matrix_eventos.pkl", "rb"))
motor_search_actividades = pickle.load(open("tfidf_motor_search_actividades.pkl", "rb"))
motor_search_matrix_actividades = pickle.load(open("tfidf_motor_search_matrix_actividades.pkl", "rb"))

def buscar_tiendas_por_nombre_y_ubicacion(df, tienda_nombre, user_location, model, modelmatrix, radius_km=5, top_n=5):
    # Calcular la similitud del coseno
    query_vector = model.transform([tienda_nombre.lower()])
    cosine_similarities = cosine_similarity(query_vector, modelmatrix).flatten()

    # Obtener los índices de las tiendas más similares
    similar_indices = cosine_similarities.argsort()[-top_n:][::-1]

    # Filtrar y ordenar las tiendas por distancia
    tiendas_similares = []
    for idx in similar_indices:
        store = df.iloc[idx]
        store_location = (store['latitud'], store['longitud'])
        distance = geodesic(user_location, store_location).km
        if distance <= radius_km:
            tiendas_similares.append((store, distance))

    # Ordenar las tiendas por distancia
    tiendas_similares.sort(key=lambda x: x[1])

    # Devolver las top n tiendas más parecidas
    top_tiendas = [store.to_dict() for store, distance in tiendas_similares[:top_n]]

    for tienda in top_tiendas:
        print(f"Tienda: {tienda['nombre']}, Distancia: {geodesic(user_location, (tienda['latitud'], tienda['longitud'])).km:.2f} km")

    return top_tiendas

user_location = (-12.135969775722781, -77.02249671518491)  # UTEC
nombre = "pizza"
tiendas_parecidas = buscar_tiendas_por_nombre_y_ubicacion(df_actividades, "caballo", user_location, motor_search_actividades, motor_search_matrix_actividades, radius_km=10, top_n=5)

print(tiendas_parecidas)

Tienda: paseo a caballo, Distancia: 4.87 km
Tienda: cata de vinos, Distancia: 5.96 km
Tienda: escalada en roca, Distancia: 6.03 km
Tienda: taller de cerámica, Distancia: 6.39 km
Tienda: taller de cerámica, Distancia: 7.42 km
[{'actividad_id': 99, 'nombre': 'paseo a caballo', 'categorias': 'Naturaleza,Aventura', 'descripcion_breve': 'Disfruta de la naturaleza con un paseo a caballo.', 'latitud': -12.124908, 'longitud': -76.979172, 'direccion': 'Dirección #99-Paseo a Caballo', 'calificaciones': 0.6, 'precio': 40, 'fecha_inicio': Timestamp('2024-10-07 00:00:00'), 'fecha_fin': Timestamp('2024-10-10 00:00:00'), 'horario_disponible': ['Sábado 14:00 - 24:00', 'Viernes 10:00 - 18:00'], 'material_incluido': True, 'target_etario': np.str_('18-65'), 'redes_sociales': 'fb.com/paseoacaballo', 'distrito': 'santiago de surco', 'tags': 'Naturaleza,Aventura'}, {'actividad_id': 96, 'nombre': 'cata de vinos', 'categorias': 'Gastronomía,Experiencia', 'descripcion_breve': 'Descubre los mejores vinos con un